In [5]:
# import libraries and datasets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load the dataset

behavior = pd.read_csv("data/MINDsmall_dev/behaviors.tsv", sep="\t", header=None, names=["Impression ID", "User ID", "Time", "History", "Impressions"])
news = pd.read_csv("data/MINDsmall_dev/news.tsv", sep="\t", header=None, names=["News ID", "Category", "Subcategory", "Title", "Abstract", "URL", "Title Entities", "Abstract Entities", "Title Topics", "Abstract Topics"])
#TODO: read embeddings

In [6]:
#Find tags of an article from an articleID
def article_tags(articleID):
    tags = []
    article = news.loc[news["News ID"] == articleID]
    tags.append(article["Category"].values[0])
    tags.append(article["Subcategory"].values[0])
    return tags[0], tags[1]

#Create user vector from a userID
def create_x_i(userID):
    x_i = {}

    # Weights
    w_c = 0.3  
    w_s = 1.0

    article_count = 0

    for index, row in behavior.iterrows():
        if row["User ID"] == userID:
            for articleID in row["History"].split(" "):
                article_count += 1

                category, subcategory = article_tags(articleID)

                if category in x_i:
                    x_i[category] += w_c
                else:
                    x_i[category] = w_c
                
                if subcategory in x_i:
                    x_i[subcategory] += w_s
                else:
                    x_i[subcategory] = w_s
    
    for trait, score in x_i.items():
        x_i[trait] = score/article_count

    return x_i

#exID = behavior["User ID"].values[1]

#x_i = create_x_i(exID)
#print(x_i)

In [7]:
#TODO create all article vectors
def create_all_x_j():
    all_x_j = []
    for index, row in news.iterrows():
        x_j = {}
        x_j["News ID"] = row["News ID"]
        x_j[row["Category"]] = 1.0
        x_j[row["Subcategory"]] = 1.0
        all_x_j.append(x_j)
    return all_x_j

#X_j = create_all_x_j()
#print(X_j[1])

In [11]:
#TODO create scoring function
def score(x_i, X_j):
    scores = {}

    for x_j in X_j:
        for c, s in x_j.items():
            article_score = 0
            if c in x_i.keys():
                article_score += s * x_i[c]
        if article_score > 0:
            scores[x_j["News ID"]] = article_score
    
    return dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

#print(score(exID))